In [1]:
# Remember you're in Conda Virtual Env: conda activate vw

# !pip install data-science-types
# !pip install tensorflow
# !pip install keras
# !pip install pydot
# !pip install graphviz
# !brew install graphviz Local Terminal for YES/NO question. # If Failure go to next line
# !xcode-select --install # If Failure go to next line
# https://stackoverflow.com/questions/19907576/xcode-is-not-currently-available-from-the-software-update-server
# -> https://developer.apple.com/download/more/?name=Command%20Line%20Tools
# -> Download top package; mine is 2020/11/05

### Seq2Seq LSTM

In [1]:
# lstm autoencoder recreate sequence
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

def reconstruct_lstm(sequence: np.ndarray):
    # reshape input into [samples, timesteps, features]
    n_in = len(sequence)
    sequence = sequence.reshape((1, n_in, 1))
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
    model.add(RepeatVector(n_in))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(sequence, sequence, epochs=300, verbose=0)
    plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
    # demonstrate recreation
    yhat = model.predict(sequence, verbose=0)
    print(yhat[0,:,0])
    return yhat[0,:,0]

# define input sequence
seq = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
reconstruct_lstm(seq)

[0.10489432 0.19891457 0.29819024 0.39916733 0.50018615 0.60077816
 0.7007725  0.8002165  0.89932036]


array([0.10489432, 0.19891457, 0.29819024, 0.39916733, 0.50018615,
       0.60077816, 0.7007725 , 0.8002165 , 0.89932036], dtype=float32)

In [3]:
# lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

def composite_lstm(seq_in: np.ndarray):

    # reshape input into [samples, timesteps, features]
    n_in = len(seq_in)
    seq_in = seq_in.reshape((1, n_in, 1))
    # prepare output sequence
    seq_out = seq_in[:, 1:, :]
    n_out = n_in - 1
    
    # define encoder
    visible = Input(shape=(n_in,1))
    encoder = LSTM(100, activation='relu')(visible)
    
    # define reconstruct decoder
    decoder1 = RepeatVector(n_in)(encoder)
    decoder1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
    decoder1 = TimeDistributed(Dense(1))(decoder1)
    
    # define predict decoder
    decoder2 = RepeatVector(n_out)(encoder)
    decoder2 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
    decoder2 = TimeDistributed(Dense(1))(decoder2)
    
    # tie it together
    model = Model(inputs=visible, outputs=[decoder1, decoder2])
    model.compile(optimizer='adam', loss='mse')
    plot_model(model, show_shapes=True, to_file='composite_lstm_autoencoder.png')
    
    # fit model
    model.fit(seq_in, [seq_in,seq_out], epochs=300, verbose=0)
    
    # demonstrate prediction
    yhat = model.predict(seq_in, verbose=0)
#     print(yhat)
    
    return yhat
    
# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
composite_lstm(seq_in)

[array([[[0.10826182],
         [0.20552245],
         [0.30258575],
         [0.40007335],
         [0.4985259 ],
         [0.59798217],
         [0.698382  ],
         [0.799459  ],
         [0.901271  ]]], dtype=float32),
 array([[[0.16565828],
         [0.28881192],
         [0.4027717 ],
         [0.50927055],
         [0.61052614],
         [0.7072726 ],
         [0.8001721 ],
         [0.8898257 ]]], dtype=float32)]

### Sequence Classification LSTM

In [1]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

17465344/17464789 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 104s 267ms/step - loss: 0.4372 - accuracy: 0.7954
Epoch 2/3
391/391 [==============================] - 103s 264ms/step - loss: 0.2909 - accuracy: 0.8823
Epoch 3/3
391/391 [==============================] - 100s 256ms/step - loss: 0.2479 - accuracy: 0.9036
A

In [14]:
print(model.predict(X_test[:3]))
# print(model.predict_classes(X_test[:3])) # Deprecated in favor of below
print( (model.predict(X_test[:3]) > 0.5).astype("int32") )

[[0.0931007]
 [0.9749841]
 [0.8212079]]
[[0]
 [1]
 [1]]


In [24]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
X_train[:3]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [17]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500


class SeqLstm:
    
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def seq2class(self, epochs=2, batch_size=100, dropout=0.2, lstm_layer=100, verbose=0):

        # Truncate or pad input sequences with 0's prior to Keras fitting
        self.X_train = sequence.pad_sequences(self.X_train, maxlen=max_review_length)
        self.X_test = sequence.pad_sequences(self.X_test, maxlen=max_review_length)

        # create the model
        embedding_vector_length = 32
        model = Sequential()
        model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
        model.add(Dropout(dropout))
        model.add(LSTM(lstm_layer))
        model.add(Dropout(dropout))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Fit Model
        model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=batch_size)

        if verbose:
            # Summary of model
            print(model.summary())
            # Final evaluation of the model
            scores = model.evaluate(self.X_test, self.y_test, verbose=0)
            print("Accuracy: %.2f%%" % (scores[1]*100))

        return model
    
ex = SeqLstm(X_train, y_train, X_test, y_test)
model = ex.seq2class(epochs=1, batch_size=200, verbose=1)

In [30]:
import numpy as np

# Demo Records
sample = X_test[:3]
for i in sample:
    print(len(i))

# Preprocess sample by padding;  The model will learn the zero values carry no information so indeed the sequences are not the same length in terms of content
new_sample = sequence.pad_sequences(sample, maxlen=max_review_length)
for i in new_sample:
    print(len(i))

# Predict probabilities and classes
print(model.predict(new_sample))
print(f"Predicted results: {(model.predict(new_sample) > 0.5).astype('int32')}" )
print(f"Actual results: {y_test[:3]}")

68
260
603
500
500
500
[[0.44343093]
 [0.5306598 ]
 [0.48315963]]
Predicted results: [[0]
 [1]
 [0]]
Actual results: [0 1 1]
